In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from googletrans import Translator
from pandas import Series
from pandas import DataFrame
%matplotlib inline


In [25]:
item_category_df = pd.read_csv("data/item_categories.csv")
items_df = pd.read_csv("data/items.csv")
shops_df = pd.read_csv("data/shops.csv")
sample_df = pd.read_csv("data/sample_submission.csv")
train_df = pd.read_csv("data/sales_train_v2.csv")
test_df = pd.read_csv("data/test.csv")

# date-> day, month, year

In [26]:
f = lambda x: x.split(".")
train_df['date'] =list(map(f, train_df['date']))

In [27]:
f = lambda x: x[0]
train_df['day'] =list(map(f, train_df['date']))

In [28]:
f = lambda x: x[1]
train_df['month'] =list(map(f, train_df['date']))

In [29]:
f = lambda x: x[2]
train_df['year'] =list(map(f, train_df['date']))

In [30]:
train_df = train_df.drop(['date'],axis=1)

In [31]:
train_df.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,day,month,year
0,0,59,22154,999.00,1.0,02,01,2013
1,0,25,2552,899.00,1.0,03,01,2013
2,0,25,2552,899.00,-1.0,05,01,2013
3,0,25,2554,1709.05,1.0,06,01,2013
4,0,25,2555,1099.00,1.0,15,01,2013


## 번역 (러시아어 -> 영어) -> tf-idf

In [32]:
trans = Translator()
def translate(column):
    en_list=[]
    for word in column:
        en = trans.translate(word, dest='en')
        en_list.append(en.text)
    return en_list

In [33]:
shops_df["shop_name"] = translate(shops_df["shop_name"])
shops_df.head()

,shop_name,shop_id
0,"Yakutsk Ordzhonikidze, 56 francs",0
1,"Yakutsk TC ""Central"" franc",1
2,"Adygea Shopping Center ""Mega""",2
3,"Balashikha TRK ""October-Kinomir""",3
4,"Volzhsky shopping center ""Volga Mall""",4


In [34]:
item_category_df["item_category_name"] = translate(item_category_df["item_category_name"])
item_category_df.head()

,item_category_name,item_category_id
0,PC - Headsets / Headphones,0
1,Accessories - PS2,1
2,Accessories - PS3,2
3,Accessories - PS4,3
4,Accessories - PSP,4


# merge

In [35]:
df =pd.merge(train_df, shops_df, on='shop_id')
df.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,day,month,year,shop_name
0,0,59,22154,999.0,1.0,02,01,2013,"Yaroslavl Shopping center ""Altair"""
1,0,59,22151,399.0,1.0,10,01,2013,"Yaroslavl Shopping center ""Altair"""
2,0,59,5603,699.0,1.0,04,01,2013,"Yaroslavl Shopping center ""Altair"""
3,0,59,5587,199.0,2.0,19,01,2013,"Yaroslavl Shopping center ""Altair"""
4,0,59,5613,5571.0,1.0,31,01,2013,"Yaroslavl Shopping center ""Altair"""


In [36]:
df =pd.merge(df, items_df, on='item_id')
df.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,day,month,year,shop_name,item_name,item_category_id
0,0,59,22154,999.0,1.0,02,01,2013,"Yaroslavl Shopping center ""Altair""",ЯВЛЕНИЕ 2012 (BD),37
1,0,25,22154,999.0,1.0,02,01,2013,"Moscow TRK ""Atrium""",ЯВЛЕНИЕ 2012 (BD),37
2,0,25,22154,999.0,1.0,03,01,2013,"Moscow TRK ""Atrium""",ЯВЛЕНИЕ 2012 (BD),37
3,0,25,22154,999.0,1.0,20,01,2013,"Moscow TRK ""Atrium""",ЯВЛЕНИЕ 2012 (BD),37
4,0,25,22154,999.0,1.0,23,01,2013,"Moscow TRK ""Atrium""",ЯВЛЕНИЕ 2012 (BD),37


In [37]:
df =pd.merge(df, item_category_df, on='item_category_id')
df.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,day,month,year,shop_name,item_name,item_category_id,item_category_name
0,0,59,22154,999.0,1.0,02,01,2013,"Yaroslavl Shopping center ""Altair""",ЯВЛЕНИЕ 2012 (BD),37,Cinema - Blu-Ray
1,0,25,22154,999.0,1.0,02,01,2013,"Moscow TRK ""Atrium""",ЯВЛЕНИЕ 2012 (BD),37,Cinema - Blu-Ray
2,0,25,22154,999.0,1.0,03,01,2013,"Moscow TRK ""Atrium""",ЯВЛЕНИЕ 2012 (BD),37,Cinema - Blu-Ray
3,0,25,22154,999.0,1.0,20,01,2013,"Moscow TRK ""Atrium""",ЯВЛЕНИЕ 2012 (BD),37,Cinema - Blu-Ray
4,0,25,22154,999.0,1.0,23,01,2013,"Moscow TRK ""Atrium""",ЯВЛЕНИЕ 2012 (BD),37,Cinema - Blu-Ray


In [38]:
# df = df.drop(['item_name'],axis=1)
# df = df.drop(['item_category_name'],axis=1)
# df = df.drop(['shop_name'],axis=1)

In [39]:
df.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,day,month,year,shop_name,item_name,item_category_id,item_category_name
0,0,59,22154,999.0,1.0,02,01,2013,"Yaroslavl Shopping center ""Altair""",ЯВЛЕНИЕ 2012 (BD),37,Cinema - Blu-Ray
1,0,25,22154,999.0,1.0,02,01,2013,"Moscow TRK ""Atrium""",ЯВЛЕНИЕ 2012 (BD),37,Cinema - Blu-Ray
2,0,25,22154,999.0,1.0,03,01,2013,"Moscow TRK ""Atrium""",ЯВЛЕНИЕ 2012 (BD),37,Cinema - Blu-Ray
3,0,25,22154,999.0,1.0,20,01,2013,"Moscow TRK ""Atrium""",ЯВЛЕНИЕ 2012 (BD),37,Cinema - Blu-Ray
4,0,25,22154,999.0,1.0,23,01,2013,"Moscow TRK ""Atrium""",ЯВЛЕНИЕ 2012 (BD),37,Cinema - Blu-Ray


# add location feature

## 동,서,남 방위와 온라인, 무역 등의 shop들로 구성됨
## 지역 위치에 따른 판매량 분석

In [40]:
loc_shop = df["shop_name"]
len(loc_shop)

2935849

In [41]:
loc = []
for i in loc_shop:
    splited_loc = i.split()
    loc.append(splited_loc[0])

In [42]:
df['location'] = loc

In [43]:
west= ['Adygea','Balashikha', 'Chekhov','Kaluga','Kazan', 'Krasnoyarsk' ,'Kursk', 'Moscow' ,'Yaroslavl',
        'Mytischi', 'N.Novgorod', 'Nizhny','RostovNaDon',  'RostovNaDonu', 'Samara' ,'SPb' ,'Sergiev', 
        'Surgut', 'Tyumen', 'Ufa', 'Vologda', 'Volzhsky', 'Voronezh' , 'Zhukovsky','Colosseum', 'Khimki']
east =  ['Yakutsk']
south =  ['Novosibirsk','Omsk', 'Tomsk']

In [44]:
s1 = Series(data = 'west', index= west)
west = s1.to_dict()

s2 = Series(data = 'east'
   , index= east)
east = s2.to_dict()

s3 = Series(data = 'south'
   , index= south)
south = s3.to_dict()

In [45]:
df["location"].replace(to_replace=west.keys(), value=west.values(), inplace=True)
df["location"].replace(to_replace=east.keys(), value=east.values(), inplace=True)
df["location"].replace(to_replace=south.keys(), value=south.values(), inplace=True)
df.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,day,month,year,shop_name,item_name,item_category_id,item_category_name,location
0,0,59,22154,999.0,1.0,02,01,2013,"Yaroslavl Shopping center ""Altair""",ЯВЛЕНИЕ 2012 (BD),37,Cinema - Blu-Ray,west
1,0,25,22154,999.0,1.0,02,01,2013,"Moscow TRK ""Atrium""",ЯВЛЕНИЕ 2012 (BD),37,Cinema - Blu-Ray,west
2,0,25,22154,999.0,1.0,03,01,2013,"Moscow TRK ""Atrium""",ЯВЛЕНИЕ 2012 (BD),37,Cinema - Blu-Ray,west
3,0,25,22154,999.0,1.0,20,01,2013,"Moscow TRK ""Atrium""",ЯВЛЕНИЕ 2012 (BD),37,Cinema - Blu-Ray,west
4,0,25,22154,999.0,1.0,23,01,2013,"Moscow TRK ""Atrium""",ЯВЛЕНИЕ 2012 (BD),37,Cinema - Blu-Ray,west


In [46]:
dummy_loc_df =pd.get_dummies(df['location'])
dummy_loc_df.head()

,Digital,Online,Outbound,east,south,west
0,0,0,0,0,0,1
1,0,0,0,0,0,1
2,0,0,0,0,0,1
3,0,0,0,0,0,1
4,0,0,0,0,0,1


In [47]:
df = pd.concat([df, dummy_loc_df],axis=1)
df = df.drop(['location'],axis=1)
df.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,day,month,year,shop_name,item_name,item_category_id,item_category_name,Digital,Online,Outbound,east,south,west
0,0,59,22154,999.0,1.0,02,01,2013,"Yaroslavl Shopping center ""Altair""",ЯВЛЕНИЕ 2012 (BD),37,Cinema - Blu-Ray,0,0,0,0,0,1
1,0,25,22154,999.0,1.0,02,01,2013,"Moscow TRK ""Atrium""",ЯВЛЕНИЕ 2012 (BD),37,Cinema - Blu-Ray,0,0,0,0,0,1
2,0,25,22154,999.0,1.0,03,01,2013,"Moscow TRK ""Atrium""",ЯВЛЕНИЕ 2012 (BD),37,Cinema - Blu-Ray,0,0,0,0,0,1
3,0,25,22154,999.0,1.0,20,01,2013,"Moscow TRK ""Atrium""",ЯВЛЕНИЕ 2012 (BD),37,Cinema - Blu-Ray,0,0,0,0,0,1
4,0,25,22154,999.0,1.0,23,01,2013,"Moscow TRK ""Atrium""",ЯВЛЕНИЕ 2012 (BD),37,Cinema - Blu-Ray,0,0,0,0,0,1


In [48]:
df = df.drop(['item_name'],axis=1)
df = df.drop(['item_category_name'],axis=1)
df = df.drop(['shop_name'],axis=1)

In [50]:
df = df.drop(['item_price'],axis=1)
df = df.drop(['day'],axis=1)
df = df.drop(['year'],axis=1)
df.head()

,date_block_num,shop_id,item_id,item_cnt_day,month,item_category_id,Digital,Online,Outbound,east,south,west
0,0,59,22154,1.0,01,37,0,0,0,0,0,1
1,0,25,22154,1.0,01,37,0,0,0,0,0,1
2,0,25,22154,1.0,01,37,0,0,0,0,0,1
3,0,25,22154,1.0,01,37,0,0,0,0,0,1
4,0,25,22154,1.0,01,37,0,0,0,0,0,1


In [68]:
len(df.columns)

12

## test

In [51]:
test_df = pd.read_csv("data/test.csv")

In [52]:
test_df.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [53]:
test_df =pd.merge(test_df, items_df, on='item_id')
test_df.head()

,ID,shop_id,item_id,item_name,item_category_id
0,0,5,5037,"NHL 15 [PS3, русские субтитры]",19
1,5100,4,5037,"NHL 15 [PS3, русские субтитры]",19
2,10200,6,5037,"NHL 15 [PS3, русские субтитры]",19
3,15300,3,5037,"NHL 15 [PS3, русские субтитры]",19
4,20400,2,5037,"NHL 15 [PS3, русские субтитры]",19


In [54]:
test_df =pd.merge(test_df, shops_df, on='shop_id')
test_df.head()

,ID,shop_id,item_id,item_name,item_category_id,shop_name
0,0,5,5037,"NHL 15 [PS3, русские субтитры]",19,"Vologda Shopping and Entertainment Center ""Mar..."
1,1,5,5320,ONE DIRECTION Made In The A.M.,55,"Vologda Shopping and Entertainment Center ""Mar..."
2,2,5,5233,"Need for Speed Rivals (Essentials) [PS3, русск...",19,"Vologda Shopping and Entertainment Center ""Mar..."
3,3,5,5232,"Need for Speed Rivals (Classics) [Xbox 360, ру...",23,"Vologda Shopping and Entertainment Center ""Mar..."
4,4,5,5268,"Need for Speed [PS4, русская версия]",20,"Vologda Shopping and Entertainment Center ""Mar..."


In [55]:
test_df =pd.merge(test_df, item_category_df, on='item_category_id')
test_df.head()

,ID,shop_id,item_id,item_name,item_category_id,shop_name,item_category_name
0,0,5,5037,"NHL 15 [PS3, русские субтитры]",19,"Vologda Shopping and Entertainment Center ""Mar...",Games - PS3
1,2,5,5233,"Need for Speed Rivals (Essentials) [PS3, русск...",19,"Vologda Shopping and Entertainment Center ""Mar...",Games - PS3
2,15,5,4872,"Minecraft. Playstation 3 Edition [PS3, русская...",19,"Vologda Shopping and Entertainment Center ""Mar...",Games - PS3
3,21,5,5002,"NBA 2K16 [PS3, русская документация]",19,"Vologda Shopping and Entertainment Center ""Mar...",Games - PS3
4,23,5,5814,"Plants vs. Zombies Garden Warfare [PS3, русска...",19,"Vologda Shopping and Entertainment Center ""Mar...",Games - PS3


In [56]:
loc_shop = test_df["shop_name"]
len(loc_shop)

214200

In [57]:
loc = []
for i in loc_shop:
    splited_loc = i.split()
    loc.append(splited_loc[0])

In [58]:
test_df['location'] = loc

In [59]:
west= ['Adygea','Balashikha', 'Chekhov','Kaluga','Kazan', 'Krasnoyarsk' ,'Kursk', 'Moscow' ,'Yaroslavl',
        'Mytischi', 'N.Novgorod', 'Nizhny','RostovNaDon',  'RostovNaDonu', 'Samara' ,'SPb' ,'Sergiev', 
        'Surgut', 'Tyumen', 'Ufa', 'Vologda', 'Volzhsky', 'Voronezh' , 'Zhukovsky','Colosseum', 'Khimki']
east =  ['Yakutsk']
south =  ['Novosibirsk','Omsk', 'Tomsk']

In [60]:
s1 = Series(data = 'west', index= west)
west = s1.to_dict()

s2 = Series(data = 'east'
   , index= east)
east = s2.to_dict()

s3 = Series(data = 'south'
   , index= south)
south = s3.to_dict()

In [61]:
test_df["location"].replace(to_replace=west.keys(), value=west.values(), inplace=True)
test_df["location"].replace(to_replace=east.keys(), value=east.values(), inplace=True)
test_df["location"].replace(to_replace=south.keys(), value=south.values(), inplace=True)
test_df.head()

,ID,shop_id,item_id,item_name,item_category_id,shop_name,item_category_name,location
0,0,5,5037,"NHL 15 [PS3, русские субтитры]",19,"Vologda Shopping and Entertainment Center ""Mar...",Games - PS3,west
1,2,5,5233,"Need for Speed Rivals (Essentials) [PS3, русск...",19,"Vologda Shopping and Entertainment Center ""Mar...",Games - PS3,west
2,15,5,4872,"Minecraft. Playstation 3 Edition [PS3, русская...",19,"Vologda Shopping and Entertainment Center ""Mar...",Games - PS3,west
3,21,5,5002,"NBA 2K16 [PS3, русская документация]",19,"Vologda Shopping and Entertainment Center ""Mar...",Games - PS3,west
4,23,5,5814,"Plants vs. Zombies Garden Warfare [PS3, русска...",19,"Vologda Shopping and Entertainment Center ""Mar...",Games - PS3,west


In [62]:
dummy_loc_df =pd.get_dummies(test_df['location'])
dummy_loc_df.head()

,Digital,Online,east,south,west
0,0,0,0,0,1
1,0,0,0,0,1
2,0,0,0,0,1
3,0,0,0,0,1
4,0,0,0,0,1


In [63]:
test_df = pd.concat([test_df, dummy_loc_df],axis=1)
test_df = test_df.drop(['location'],axis=1)
test_df.head()

,ID,shop_id,item_id,item_name,item_category_id,shop_name,item_category_name,Digital,Online,east,south,west
0,0,5,5037,"NHL 15 [PS3, русские субтитры]",19,"Vologda Shopping and Entertainment Center ""Mar...",Games - PS3,0,0,0,0,1
1,2,5,5233,"Need for Speed Rivals (Essentials) [PS3, русск...",19,"Vologda Shopping and Entertainment Center ""Mar...",Games - PS3,0,0,0,0,1
2,15,5,4872,"Minecraft. Playstation 3 Edition [PS3, русская...",19,"Vologda Shopping and Entertainment Center ""Mar...",Games - PS3,0,0,0,0,1
3,21,5,5002,"NBA 2K16 [PS3, русская документация]",19,"Vologda Shopping and Entertainment Center ""Mar...",Games - PS3,0,0,0,0,1
4,23,5,5814,"Plants vs. Zombies Garden Warfare [PS3, русска...",19,"Vologda Shopping and Entertainment Center ""Mar...",Games - PS3,0,0,0,0,1


In [64]:
test_df = test_df.drop(['item_name'],axis=1)
test_df = test_df.drop(['item_category_name'],axis=1)
test_df = test_df.drop(['shop_name'],axis=1)
test_df.head()

,ID,shop_id,item_id,item_category_id,Digital,Online,east,south,west
0,0,5,5037,19,0,0,0,0,1
1,2,5,5233,19,0,0,0,0,1
2,15,5,4872,19,0,0,0,0,1
3,21,5,5002,19,0,0,0,0,1
4,23,5,5814,19,0,0,0,0,1


In [66]:
test_df['month'] =11
test_df['date_block_num']= 34
test_df.head()

,ID,shop_id,item_id,item_category_id,Digital,Online,east,south,west,month,date_block_num
0,0,5,5037,19,0,0,0,0,1,11,34
1,2,5,5233,19,0,0,0,0,1,11,34
2,15,5,4872,19,0,0,0,0,1,11,34
3,21,5,5002,19,0,0,0,0,1,11,34
4,23,5,5814,19,0,0,0,0,1,11,34


In [67]:
len(test_df.columns)

11

## ==============전처리 끝===================

In [69]:
y_data = df.item_cnt_day.reshape(df.item_cnt_day.size,1)

/home/rosa/miniconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """Entry point for launching an IPython kernel.


In [82]:
y_data

array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [2.],
       [1.]])

In [70]:
df = df.drop('item_cnt_day',axis =1)

In [71]:
x_data = df.values

In [83]:
x_data

array([[0, 59, 22154, ..., 0, 0, 1],
       [0, 25, 22154, ..., 0, 0, 1],
       [0, 25, 22154, ..., 0, 0, 1],
       ...,
       [32, 55, 13094, ..., 0, 0, 0],
       [32, 55, 13094, ..., 0, 0, 0],
       [33, 55, 13092, ..., 0, 0, 0]], dtype=object)

In [72]:
x_data.shape

(2935849, 11)

In [73]:
from sklearn import preprocessing

minmax_scale = preprocessing.MinMaxScaler(feature_range=(0,5)).fit(x_data)
x_scaled_data = minmax_scale.transform(x_data)


/home/rosa/miniconda3/envs/py36/lib/python3.6/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


array([[0.        , 5.        , 4.9966169 , 0.        , 2.22891566,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        5.        ],
       [0.        , 2.11864407, 4.9966169 , 0.        , 2.22891566,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        5.        ],
       [0.        , 2.11864407, 4.9966169 , 0.        , 2.22891566,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        5.        ]])

In [74]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x_scaled_data, y_data, test_size=0.2)

In [75]:
y_data.shape

(2935849, 1)

In [76]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1967018, 11), (968831, 11), (1967018, 1), (968831, 1))

In [77]:
from sklearn import  linear_model

regr = linear_model.LinearRegression(n_jobs=1)
regr.fit(X_train, y_train)
regr

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [78]:
print('Coefficients: ', regr.coef_) # w1
print('intercept: ', regr.intercept_) #w0

Coefficients:  [[ 3.38952288e-03 -1.22587513e-03  2.81703388e-02  2.48752127e-02
   2.23033037e-02 -6.00182746e+11 -6.00182746e+11 -6.00182746e+11
  -6.00182746e+11 -6.00182746e+11 -6.00182746e+11]]
intercept:  [3.00091373e+12]


# Metric 측정

In [79]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [80]:
y_true = y_test
y_hat = regr.predict(X_test)

r2_score(y_true, y_hat), mean_absolute_error(y_true, y_hat), mean_squared_error(y_true, y_hat)
# r2_score : 1에 가까우면 좋음  

(0.004763450414835391, 0.43941042846917444, 5.787528036691839)

In [81]:
y_true = y_train
y_hat = regr.predict(X_train)

r2_score(y_true, y_hat), mean_absolute_error(y_true, y_hat), mean_squared_error(y_true, y_hat)

(0.004099345252627229, 0.4414592398219507, 7.341817634478245)

In [1]:
#X_train, X_test, y_train, y_test = train_test_split(x_scaled_data, y_data, test_size=0.2)
#(0.004763450414835391, 0.43941042846917444, 5.787528036691839)
#(0.004099345252627229, 0.4414592398219507, 7.341817634478245)